In [4]:
from pyspark import SparkContext

In [5]:
sc = SparkContext(master='local[*]', appName='Book')

In [6]:
book_rdd = sc.textFile('/content/Books.csv')
user_rdd = sc.textFile('/content/Users.csv')
rate_rdd = sc.textFile('/content/Book-Ratings.csv')

In [7]:
def Remove_header(data):
  header=data.first()
  return data.filter(lambda x: x != header)

In [8]:
def CleanData(data,rate):
    parts = data.split(";")
    if(rate==0):
      return (parts[0].strip('"'), parts[1].strip('"'))
    else:
      return (parts[0].strip('"'), parts[1].strip('"'),float(parts[2].strip('"')))

In [9]:
book_clean=Remove_header(book_rdd).map(lambda x: CleanData(x,0))
book_clean.take(5)

[('0195153448', 'Classical Mythology'),
 ('0002005018', 'Clara Callan'),
 ('0060973129', 'Decision in Normandy'),
 ('0374157065',
  'Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It'),
 ('0393045218', 'The Mummies of Urumchi')]

In [10]:
user_clean=Remove_header(user_rdd).map(lambda x: CleanData(x,0))
user_clean.take(5)

[('1', 'bzsufoRTLN2'),
 ('2', 'fq7kfHg4VEI'),
 ('3', 'W0Hbkd3xR8v'),
 ('4', 'W51GahAx5Ap'),
 ('5', 'VKN3PQ18GgN')]

In [11]:
rate_clean=Remove_header(rate_rdd).map(lambda x: CleanData(x,1))
rate_clean.take(5)

[('276725', '034545104X', 0.0),
 ('276726', '0155061224', 5.0),
 ('276727', '0446520802', 0.0),
 ('276729', '052165615X', 3.0),
 ('276729', '0521795028', 6.0)]

In [12]:
def Avg_Book(data):
  book_rate=data.map(lambda x: (x[1],(x[2],1)))
  reduce_sum_count=book_rate.reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))
  return reduce_sum_count.map(lambda x: (x[0],x[1][0] / x[1][1]))

In [13]:
avg_book=Avg_Book(rate_clean)
avg_book.take(5)

[('034545104X', 2.933333333333333),
 ('0155061224', 2.5),
 ('0521795028', 6.0),
 ('0600570967', 6.0),
 ('342310538', 5.0)]

In [50]:
isbnKey_Rate=rate_clean.map(lambda x: (x[1],(x[0],x[2])))
bookName_avgRate=isbnKey_Rate.join(avg_book).join(book_clean)
bookName_avgRate.first()

('0060517794',
 ((('276747', 9.0), 3.6363636363636362), 'Little Altars Everywhere'))

In [59]:
result1=bookName_avgRate.map(lambda x: (x[1][0][0][0],x[1][1],x[1][0][0][1],x[1][0][1]))
result1.take(5)

[('276747', 'Little Altars Everywhere', 9.0, 3.6363636363636362),
 ('278843', 'Little Altars Everywhere', 7.0, 3.6363636363636362),
 ('4017', 'Little Altars Everywhere', 10.0, 3.6363636363636362),
 ('7158', 'Little Altars Everywhere', 0.0, 3.6363636363636362),
 ('8961', 'Little Altars Everywhere', 8.0, 3.6363636363636362)]

In [60]:
userName_result1=result1.map(lambda x: (x[0],(x[1],x[2],x[3]))).join(user_clean)
userName_result1.first()

('53281',
 (('Little Altars Everywhere', 0.0, 3.6363636363636362), 'qH7HZ4GG8yU'))

In [64]:
finalResult=userName_result1.map(lambda x: (x[1][1],x[1][0][0],x[1][0][1],x[1][0][2]))
finalResult.take(5)

[('qH7HZ4GG8yU', 'Little Altars Everywhere', 0.0, 3.6363636363636362),
 ('qH7HZ4GG8yU', 'Nine Months', 0.0, 1.875),
 ('qH7HZ4GG8yU', 'My Cousin Rachel (Common Reader Editions)', 0.0, 3.8),
 ('qH7HZ4GG8yU', 'Mesmerized', 0.0, 2.2142857142857144),
 ('qH7HZ4GG8yU', 'The Magic of Ordinary Days', 8.0, 3.7142857142857144)]

In [65]:
finalResult.saveAsTextFile('/content/bookRate_output')